# Processamento do Anexo C

In [1]:
import pandas as pd
from dotenv import load_dotenv
import os
import re
from src.anexo_c.process_dashboard import process_dashboard
from src.anexo_c.export_pdf import export_pdf
from tqdm.auto import tqdm

## Configurações

In [2]:
load_dotenv()

DASHBOARD_CSV = r"data\input\Diligencia_Contratos.csv"
TEMPLATE_PATH = r"docs\template_csll.xlsx"
TAX_FILTERS = {
    "IRPJ": ["RAIR", "Exclusão", "Adição"],
    "CS": ["RAIR", "Exclusão", "Adição"],
}

INPUT_DIR = r"data\input"
OUTPUT_DIR = r"data\output"
INVALID_DIR = r"data\invalid"

VALUE_COLS = ["ValorDebito", "ValorCredito", "Movimentacao"]
GROUP_COLS = ["Conta_Nome", "Cosif_Nome"]

CONFIGS = [
    (
        {"IRPJ": ["RAIR", "Exclusão", "Adição"], "CS": ["RAIR", "Exclusão", "Adição"]},
        "IRPJ",
        "Resultado antes do IR",
    ),
    ({"IRPJ": ["Adição"], "CS": ["RAIR", "Exclusão", "Adição"]}, "IRPJ", "Adição"),
    ({"IRPJ": ["Exclusão"], "CS": ["RAIR", "Exclusão", "Adição"]}, "IRPJ", "Exclusão"),
    (
        {"IRPJ": ["RAIR", "Exclusão", "Adição"], "CS": ["RAIR", "Exclusão", "Adição"]},
        "CSLL",
        "Resultado antes do CSLL",
    ),
    ({"IRPJ": ["RAIR", "Exclusão", "Adição"], "CS": ["Adição"]}, "CSLL", "Adição"),
    ({"IRPJ": ["RAIR", "Exclusão", "Adição"], "CS": ["Exclusão"]}, "CSLL", "Exclusão"),
]

## Geração do Excel

In [3]:
dashboard = pd.read_csv(DASHBOARD_CSV, sep=';', encoding='latin1')

In [4]:
contratos = dashboard['NumContrato'].unique().tolist()
contratos_selecionados = pd.read_excel(r"docs\contratos_com_erro.xlsx")['arquivo'].str.replace(".xlsx", "").str.replace("C","").astype(int).sort_values().tolist()
contratos_to_process = [c for c in contratos if c in contratos_selecionados]

In [5]:
cls = process_dashboard(TAX_FILTERS, TEMPLATE_PATH)

In [22]:
cls.get_contract_numbers(OUTPUT_DIR)

✅ Extraídos 0 itens e salvos em 'numeros_extraidos.txt'


In [23]:
nao_encontrados_3 = set()
if os.path.isfile("numeros_extraidos.txt"):
    with open("numeros_extraidos.txt", "r") as f:
        nao_encontrados_3 = set(line.strip() for line in f)


In [25]:

# Filtre os contratos removendo aqueles que estão na lista de não encontrados
contratos = [
    int(contrato)
    for contrato in contratos_to_process
    if str(contrato) not in nao_encontrados_3
]


In [ ]:
final_final = pd.DataFrame()
for contrato in tqdm(contratos, unit=" contrato"):
    tqdm.desc = f"processando contrato: {contrato}"
    dashboard_filtrado = dashboard[dashboard["NumContrato"] == contrato]

    dashboard_filtro = cls.replicate_years(
        dashboard_filtrado,  tax_cols=["IRPJ", "CS"]
    )
    final = pd.DataFrame()

    # para cada filtro, gera o bloco e concatena
    for filtros, secao, desc in CONFIGS:
        bloco = cls.group_revenues(dashboard_filtro, filtros, secao, desc)
        final = pd.concat([final, bloco], ignore_index=True)

    final = cls.calcula_csll(final)
    df = final.copy()
    df['NumContrato'] = contrato
    final_final = pd.concat([final_final, df])
    
    # num_str = str(contrato)
    # num_str = num_str.zfill(7)      # completa com zeros à esquerda
    # output_path = os.path.join(OUTPUT_DIR, f"C{num_str}.xlsx")

    # cls.atualizar_template_pivot(
    #     template_path=TEMPLATE_PATH, output_path=output_path, df=df, contrato=contrato, new_pivot_name="IR_CS_ANUAL"
    # )

  0%|          | 0/10157 [00:00<?, ?contrato/s]

KeyboardInterrupt: 

## Exportação do PDF

In [ ]:
final_final.to_csv("final.csv")

In [ ]:
#export_pdf().process_file()

# ( •̀ ω •́ )y